In [39]:
# Install required packages for the RAG system
%pip install langchain-ollama langchain langchain-community faiss-cpu langchain_huggingface rank_bm25 gradio nest_asyncio markdown2 ipywidgets langchain_openai sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Chunks and Embedding loader 

In [40]:
import pickle
import os
from typing import List
from langchain.schema import Document

def load_chunks_from_disk(chunks_path: str) -> List[Document]:
    print(f"\nLoading chunks from {chunks_path}...")
    
    # Check if file exists
    if not os.path.exists(chunks_path):
        raise FileNotFoundError(f"Chunks file not found at {chunks_path}")
    
    # Load the chunks from disk
    with open(chunks_path, "rb") as f:
        chunks = pickle.load(f)
    
    print(f"Loaded {len(chunks)} chunks from disk")
    return chunks

In [41]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

loaded_faiss_store = FAISS.load_local(
    "/workspaces/RAG_BOT/LocalEmbeddings/Chatgpt_Enriched_Full_Embedding",
    embedding_model,
    allow_dangerous_deserialization=True
)
print("FAISS vector store loaded successfully.")

chunks = load_chunks_from_disk("/workspaces/RAG_BOT/LocalChunks/document_chunks_20250719_091443.pkl")
print("Chunks loaded successfully.")

FAISS vector store loaded successfully.

Loading chunks from /workspaces/RAG_BOT/LocalChunks/document_chunks_20250719_091443.pkl...
Loaded 683 chunks from disk
Chunks loaded successfully.


## LLM Configuration

In [42]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_openai import ChatOpenAI
from getpass import getpass

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,
    check_every_n_seconds=0.1,
    max_bucket_size=10,
)

openai_api_key = getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key = openai_api_key,
    temperature=0.1,
    rate_limiter=rate_limiter
)

# RETRIEVAL

## Setup retrievers

In [43]:
from langchain_community.retrievers import BM25Retriever

try:
    all_docs = [loaded_faiss_store.docstore._dict[doc_id] for doc_id in loaded_faiss_store.index_to_docstore_id.values()]
except AttributeError:
    all_docs = [loaded_faiss_store.docstore.get(doc_id) for doc_id in loaded_faiss_store.index_to_docstore_id.values()]

bm25_retriever = BM25Retriever.from_documents(all_docs)
bm25_retriever.k = 2

sst_retriever = loaded_faiss_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 2}
)

## Set up prompt templates


In [44]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

SYSTEM_PROMPT = """
You are a highly knowledgeable and helpful CyberArk API documentation assistant. Your primary role is to answer developer questions accurately and clearly, *using only the provided API documentation context*.

**GENERAL RULES FOR ALL RESPONSES:**
1.  **Context Reliance:** Answer *ONLY* based on the provided "Documentation Context". Do not use external knowledge or invent information.
2.  **Handling Missing Information:** If the answer to the user's question is not explicitly found within the provided documentation context, politely state: "I don't have that specific information in the documentation I can access." Do NOT guess or invent details.
3.  **Markdown Formatting:** Always use Markdown for structuring your answers (headers, code blocks, bullet points, etc.) to enhance readability.


**SPECIFIC RESPONSE BEHAVIORS:**

* **IF the user input is a general greeting (e.g., "hello", "hi", "hey"):**
    * Respond politely as a friendly assistant.
    * Example: "Hello! I'm your CyberArk API assistant. How can I help you with the CyberArk API today?"

* **ELSE IF the user is asking about a specific API endpoint:**
    * Provide detailed endpoint information. This should include:
        * Path and HTTP method (GET, POST, PUT, DELETE)
        * Required parameters (query, path, body)
        * Security requirements
        * Request body schema (in JSON if available)
        * Response body schema (in JSON if available)
        * Any available sample requests and responses.
    * Format your response with markdown, using headers for sections and code blocks for JSON examples.

* **ELSE IF the user is asking a general question about CyberArk API functionality (not tied to a single endpoint):**
    * Answer based ONLY on the provided context.
    * Try to be clear, concise, informative and straight to the point.
    * Include relevant code examples only if it is asked in query.

* **ELSE IF the Documentation Context is not available or question is outside the scope of CyberArk API documentation:**
    * Politely state: "I'm specialized in CyberArk API documentation. I don't have information about that topic in my knowledge base."
"""


system_message = SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT)

human_message = HumanMessagePromptTemplate.from_template(
    """
You are answering questions about CyberArk's API. Use the documentation context

Documentation Context:
----------------------
{context}

New User Question:
----------------------
{question}
"""
)

## Neighbourhood Expansion

In [45]:
def find_common_documents(docs1, docs2):
    if not docs1 and not docs2:
        return []
    if not docs1:
        return [docs2[0]] if docs2 else []
    if not docs2:
        return [docs1[0]] if docs1 else []
    
    doc_identifiers = {(doc.metadata.get('doc_index'), doc.metadata.get('chunk_index')) 
                      for doc in docs1 
                      if 'doc_index' in doc.metadata and 'chunk_index' in doc.metadata}
    
    common_docs = [doc for doc in docs2 
                  if 'doc_index' in doc.metadata and 'chunk_index' in doc.metadata and
                  (doc.metadata.get('doc_index'), doc.metadata.get('chunk_index')) in doc_identifiers]
    
    if not common_docs:
        return [docs1[0], docs2[0]]
    
    return common_docs


In [46]:
from typing import List, Dict, Any

def get_adjacent_docs(chunks: List[Document], doc_index: int, chunk_index_in_doc: int, doc_name: str, n: int = 1, debug = False) -> Dict[str, Any]:
    doc_chunks = [
        chunk for chunk in chunks 
        if chunk.metadata.get("doc_index") == doc_index and
        chunk.metadata.get("doc_name") == doc_name
    ]
    
    if not doc_chunks:
        raise ValueError(f"No chunks found for document index {doc_index}")
    
    doc_chunks.sort(key=lambda x: x.metadata.get("chunk_index"))
    
    target_chunk = None
    current_position = -1
    
    for i, chunk in enumerate(doc_chunks):
        if chunk.metadata.get("chunk_index") == chunk_index_in_doc:
            target_chunk = chunk
            current_position = i
            break
    
    if current_position == -1:
        raise ValueError(f"Chunk with index {chunk_index_in_doc} not found in document {doc_index}")
    
    if debug:
        print(f"\nRetrieving adjacent chunks for document {doc_index}:")
        print(f"  Document name: {target_chunk.metadata.get('doc_name', 'Unknown')}")
        print(f"  Chunk position within document: {chunk_index_in_doc + 1} of {target_chunk.metadata.get('total_chunks_in_doc', 'Unknown')}")
        print(f"  Retrieving {n} chunks before and after")
    
    prev_chunks = []
    start_idx = max(0, current_position - n)
    if start_idx < current_position:
        prev_chunks = doc_chunks[start_idx:current_position]
    
    next_chunks = []
    end_idx = min(len(doc_chunks), current_position + n + 1)
    if current_position + 1 < end_idx:
        next_chunks = doc_chunks[current_position + 1:end_idx]
    
    print(f"  Found {len(prev_chunks)} previous chunks and {len(next_chunks)} next chunks")
    
    return {
        "current_chunk": target_chunk,
        "prev_chunks": prev_chunks,
        "next_chunks": next_chunks,
        "all_doc_chunks": doc_chunks
    }

In [55]:
def retrieve_docs_with_neighborhood_expansion(query, debug=False):

    similarity_docs = sst_retriever.invoke(query)
    bm25_docs = bm25_retriever.invoke(query)

    relevant_docs = find_common_documents(similarity_docs, bm25_docs)
    final_expanded_docs = []
    if len(relevant_docs)>0:
        for doc in relevant_docs:
            chunk_index = doc.metadata.get('chunk_index')
            doc_index = doc.metadata.get('doc_index')
            doc_name = doc.metadata.get('doc_name')
            response = get_adjacent_docs(chunks, doc_index, chunk_index, doc_name, n=2, debug=debug)
            expanded_docs = response["prev_chunks"] + [response["current_chunk"]] + response["next_chunks"]
            final_expanded_docs.extend(expanded_docs)
            
    return final_expanded_docs

## Multiple Query Generation + BERT-Reranking Retriever

### Get required one line descriptions using the BM25 Retriver

In [48]:
def get_descriptions_with_keyword_matching(query):

    # 1. Retrieve documents using BM25 retriever
    retrieved_docs = bm25_retriever.invoke(query)
    
    # 2. Get top 2 documents
    top_docs = retrieved_docs[:2] if len(retrieved_docs) >= 2 else retrieved_docs
    
    # 3. Load OneLiners.json
    import json
    with open("/workspaces/RAG_BOT/Truths/OneLiners.json", "r") as f:
        one_liners_by_category = json.load(f)
    
    # 4. Process each document and find matching one-liners
    categories_found = []
    for doc in top_docs:
        # Extract category from doc_name
        category = "Unknown"
        
        if 'doc_name' in doc.metadata:
            doc_name = doc.metadata['doc_name']
            # Split by underscore and take the first part as category
            parts = doc_name.split('_')
            if parts:
                category = parts[0]        
        
        categories_found.append(category)

    categories_found = list(set(categories_found))

    # Build formatted results with category headers
    formatted_results = []
    for category in categories_found:
        formatted_results.append(f"## {category}")
        formatted_results.extend(one_liners_by_category[category])
    
    return formatted_results

### Generate subqueries by passing one line context

In [49]:
from langchain_core.prompts import PromptTemplate


def generate_subqueries(query: str):

    # Get API context from keyword matching
    api_context = get_descriptions_with_keyword_matching(query)
    api_context_text = "\n".join(api_context)
    
    # Create the prompt for the LLM
    subquery_prompt = PromptTemplate.from_template("""
    You are a helpful assistant working with CyberArk API documentation. Given a user's natural language query, your task is to break it down into smaller, API-relevant subqueries. These subqueries should help clarify what exact information or API functionality the user is asking about, based on the available API endpoints.

    ## This is a detailed breakdown of all the available endpoints use this a context:
    {api_context_text}

    ## Guidelines:
    - Return 2 to 3 **clear and atomic** subqueries.
    - Each subquery should help narrow down or clarify user intent.
    - Focus strictly on what's possible based on the API spec.
    - Include any possible synonyms or related terms.
    - Include common ways developers might phrase this question.
    - Optionally incorporate relevant concepts such as endpoint names, paths, HTTP methods, parameters, request or response schemas if they are likely relevant.

    ### User Query:
    {query}

    ### Output format:
    - You should output 2-3 subqueries with comma seperation.
    - Example Output Format: sub_query1, subquery2, subquery3                                        
    """)
    
      # Generate evaluation response
    generation_chain = subquery_prompt | llm
    subquery_response = generation_chain.invoke({
        "api_context_text": api_context_text,
        "query" : query
    })

    # Extract content if using ChatOpenAI
    if hasattr(subquery_response, 'content'):
        subquery_response = subquery_response.content
    
    
    # First, try splitting by comma
    subqueries = [sq.strip() for sq in subquery_response.split(',')]

    subqueries.append(query) # Ensure original query is included
    
    return subqueries

### Retrive Docs with query expansion
1. Generates multiple subqueries from the user's query
2. Retrieves documents for each subquery using BM25 and similarity search
3. Combines all results and removes duplicates based on document metadata

In [50]:
def retrieve_documents_with_query_expansion(query: str, debug: bool = False):

    if debug:
        print(f"Processing query: '{query}'")
    
    # Generate subqueries
    subqueries = generate_subqueries(query)
    
    if debug:
        print(f"Generated {len(subqueries)} subqueries:")
        for i, subq in enumerate(subqueries):
            print(f"  {i+1}. {subq}")
    
    # Initialize empty list to collect all documents
    all_docs = []
    
    # Process each subquery
    for i, subquery in enumerate(subqueries):
        if debug:
            print(f"Processing subquery {i+1}/{len(subqueries)}: '{subquery}'")
        
        # Get documents from BM25 retriever
        bm25_docs = bm25_retriever.invoke(subquery)
        if debug:
            print(f"BM25 retriever found {len(bm25_docs)} documents")
        
        # Get documents from similarity search retriever
        sst_docs = sst_retriever.invoke(subquery)
        if debug:
            print(f"Similarity retriever found {len(sst_docs)} documents")
        
        # Add all retrieved documents to our collection
        all_docs.extend(bm25_docs)
        all_docs.extend(sst_docs)
    
    if debug:
        print(f"Total documents retrieved: {len(all_docs)} (before deduplication)")
    
    # Remove duplicates based on doc_index and chunk_index
    unique_docs = []
    seen_indices = set()
    
    for doc in all_docs:
        # Extract document and chunk indices
        doc_index = doc.metadata.get('doc_index')
        chunk_index = doc.metadata.get('chunk_index')
        
        # Skip documents without proper metadata
        if doc_index is None or chunk_index is None:
            if debug:
                print(f"Skipping document with missing metadata: {doc.metadata}")
            continue
        
        # Create a unique identifier for this document chunk
        doc_identifier = (doc_index, chunk_index)
        
        # Add to unique docs if we haven't seen it before
        if doc_identifier not in seen_indices:
            seen_indices.add(doc_identifier)
            unique_docs.append(doc)
    
    if debug:
        print(f"After deduplication: {len(unique_docs)} unique documents")
        if len(unique_docs) > 0:
            # Show some information about the first few documents
            print("Sample of retrieved documents:")
            for i, doc in enumerate(unique_docs):
                doc_name = doc.metadata.get('doc_name', 'Unknown')
                print(f"  Document {i+1}: {doc_name} (doc_index={doc.metadata.get('doc_index')}, chunk_index={doc.metadata.get('chunk_index')})")
                content_preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
                print(f"  Preview: {content_preview}\n")
    
    return unique_docs

### BERT based reordering 
Retrieve and reorder documents using BERT embeddings to prioritize by semantic similarity to the query. Only includes documents with similarity score > 0.2.

In [51]:
from sentence_transformers import SentenceTransformer
import numpy as np
def get_bert_reordered_documents(query: str, debug: bool = False):
    if debug:
        print(f"Processing query: '{query}'")
    
    # Get documents using query expansion
    retrieved_docs = retrieve_documents_with_query_expansion(query, debug=debug)
    
    if debug:
        print(f"Retrieved {len(retrieved_docs)} documents")
    
    if not retrieved_docs:
        return []
        
    if debug:
        print("Loading BERT model for semantic reordering")
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Using the same model as your embeddings
    
    # Extract document contents
    doc_texts = [doc.page_content for doc in retrieved_docs]
    
    # Compute embeddings
    if debug:
        print("Computing BERT embeddings for query and documents")
    query_embedding = model.encode([query])[0]
    doc_embeddings = model.encode(doc_texts)
    
    # Compute similarities
    similarities = []
    for i, doc_embedding in enumerate(doc_embeddings):
        similarity = np.dot(query_embedding, doc_embedding) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
        )
        similarities.append((i, similarity))
    
    # Filter documents with similarity score above threshold (0.2)
    threshold = 0.2
    filtered_similarities = [(idx, score) for idx, score in similarities if score > threshold]
    
    if debug:
        print(f"Filtering documents with similarity score > {threshold}")
        print(f"Before filtering: {len(similarities)} documents")
        print(f"After filtering: {len(filtered_similarities)} documents")
    
    # Sort documents by similarity (highest first)
    filtered_similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Reorder documents
    reordered_docs = [retrieved_docs[idx] for idx, _ in filtered_similarities]
    
    if debug:
        print("Documents reordered by semantic similarity to query")
        for i, (idx, score) in enumerate(filtered_similarities):
            doc = retrieved_docs[idx]
            print(f"  {i+1}. [{score:.4f}] {doc.metadata.get('doc_name', 'Unknown')}")
        
        if len(filtered_similarities) == 0:
            print("  No documents met the similarity threshold. Consider lowering the threshold.")
    
    return reordered_docs

## Recursive Retrieval-Generation with Self-Correction

### Identify the gaps

In [52]:
from langchain_core.prompts import PromptTemplate
import re


def identify_knowledge_gaps(query: str, initial_response: str):

    # Create the prompt for gap identification
    gap_prompt = PromptTemplate.from_template("""
    You are analyzing a response to a user question about the CyberArk API. Your goal is to identify any information gaps, 
    uncertainties, or areas where more specific details would improve the answer.
    
    Original Question: {query}
    
    Response to Analyze: {response}
    
    Think about:
    1. Missing technical details that would make the answer more complete
    2. Ambiguous statements that need clarification
    3. API parameters, options, or behaviors not fully explained
    4. Missing examples or use cases that would improve understanding
    5. Areas where the answer expresses uncertainty or lack of information
    
    For each gap you identify, formulate a specific follow-up query that would help retrieve the missing information.
    Return a list of 1-3 follow-up queries, formatted as a comma-separated list.
    
    If the response seems comprehensive and doesn't have significant gaps, return "COMPLETE".
    
    Follow-up Queries:
    """)
    
    # Generate the gap analysis
    generation_chain = gap_prompt | llm
    gap_response = generation_chain.invoke({
        "query": query,
        "response": initial_response
    })
    
    # Extract content if using ChatOpenAI
    if hasattr(gap_response, 'content'):
        gap_response = gap_response.content
    
    # If the LLM thinks the response is complete, return empty list
    if "COMPLETE" in gap_response:
        return []
    
    # Split the follow-up queries
    follow_up_queries = [q.strip() for q in gap_response.split(',')]
    return follow_up_queries

```markdown
RAG Pipeline
├── Input: User Question
├── Check: Is it a Greeting?
│   ├── Yes → Return greeting response
│   └── No → Continue with BERT + LLM Chain
│       ├── Step 1: bert_retrieval_runnable
│       │   └── Retrieves documents using BERT reordering
│       ├── Step 2: initial_generation_chain
│       │   ├── Input: Context created from initial_docs
│       │   └── Output: initial_response from LLM
│       ├── Step 3: identify_knowledge_gaps
│       │   └── Uses LLM to extract follow-up questions based on gaps
│       ├── Check: Any follow-up questions?
│       │   ├── No → Return initial_response
│       │   └── Yes → Continue with follow-up reasoning
│       │       ├── Step 4: retrieve_docs_with_neighborhood_expansion
│       │       │   └── For each follow-up, retrieves more context
│       │       ├── Step 5: Combine initial and follow-up context
│       │       ├── Step 6: final_generation_prompt + LLM
│       │       │   └── Generates enhanced answer using all context
│       │       └── Output: final_response
└── Output: Response to user
```


In [53]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
def get_answer_with_recursive_generation(query: str, debug=False):
    """
    Implement the recursive retrieval-generation technique.
    
    Args:
        query: User question
        debug: Whether to print debug information
        
    Returns:
        Enhanced response from the LLM
    """
    # Skip retrieval for simple greetings
    greeting_terms = ["hello", "hi", "hey", "greetings"]
    is_greeting = any(term in query.lower() for term in greeting_terms) and len(query.split()) < 4
    
    if is_greeting:
        return "Hello! I'm your CyberArk API assistant. How can I help you with the CyberArk API today?"
    
    # STEP 1: Initial retrieval using an existing method (choose the best one as base)
    # Let's use BERT reranking as our base method
    if debug:
        print(f"STEP 1: Initial retrieval for query: '{query}'")
    
    initial_docs = get_bert_reordered_documents(query, debug=debug)
    initial_context = "\n\n".join([doc.page_content for doc in initial_docs])
    
    if debug:
        print(f"Retrieved {len(initial_docs)} documents for initial context")
    
    # STEP 2: Generate initial response
    if debug:
        print("STEP 2: Generating initial response")
    
    chat_prompt = ChatPromptTemplate.from_messages([
        system_message,
        HumanMessagePromptTemplate.from_template(
        """
        You are answering questions about CyberArk's API. Use the documentation context

        Documentation Context:
        ----------------------
        {context}

        New User Question:
        ----------------------
        {question}
        """
        )
    ])
    
    generation_chain = chat_prompt | llm
    initial_response = generation_chain.invoke({
        "context": initial_context,
        "question": query
    })
    
    if hasattr(initial_response, 'content'):
        initial_response = initial_response.content
    
    # STEP 3: Identify knowledge gaps
    if debug:
        print("STEP 3: Analyzing initial response for knowledge gaps")
    
    follow_up_queries = identify_knowledge_gaps(query, initial_response)
    
    if not follow_up_queries:
        if debug:
            print("No knowledge gaps identified. Returning initial response.")
        return initial_response
    
    if debug:
        print(f"Identified {len(follow_up_queries)} knowledge gaps:")
        for i, fq in enumerate(follow_up_queries):
            print(f"  {i+1}. {fq}")
    
    # STEP 4: Retrieve additional context for follow-up queries
    if debug:
        print("STEP 4: Retrieving additional context for follow-up queries")
    
    additional_docs = []
    for follow_up in follow_up_queries:
        # Use a different retrieval method for diversity
        follow_up_docs = retrieve_docs_with_neighborhood_expansion(follow_up)
        additional_docs.extend(follow_up_docs)
    
    # Remove duplicates
    seen_indices = set()
    unique_additional_docs = []
    
    for doc in additional_docs:
        doc_index = doc.metadata.get('doc_index')
        chunk_index = doc.metadata.get('chunk_index')
        
        if doc_index is None or chunk_index is None:
            continue
        
        doc_identifier = (doc_index, chunk_index)
        
        if doc_identifier not in seen_indices:
            seen_indices.add(doc_identifier)
            unique_additional_docs.append(doc)
    
    if debug:
        print(f"Retrieved {len(unique_additional_docs)} additional documents")
    
    # STEP 5: Generate enhanced response with combined context
    if debug:
        print("STEP 5: Generating enhanced response with combined context")
    
    # Combine initial and additional context
    additional_context = "\n\n".join([doc.page_content for doc in unique_additional_docs])
    combined_context = f"{initial_context}\n\n{additional_context}"
    
    # Create prompt for final generation
    enhanced_prompt = ChatPromptTemplate.from_messages([
        system_message,
        HumanMessagePromptTemplate.from_template(
        """
        You are answering questions about CyberArk's API. Use the documentation context below.
        
        This context includes both initial information and additional details to provide a more complete answer.

        Documentation Context:
        ----------------------
        {context}

        User Question:
        ----------------------
        {question}
        
        Your previous response had some gaps or uncertainties. With the additional context, provide an enhanced, 
        more complete response that addresses those gaps while maintaining accuracy and clarity.
        """
        )
    ])
    
    # Generate enhanced response
    enhanced_generation_chain = enhanced_prompt | llm
    enhanced_response = enhanced_generation_chain.invoke({
        "context": combined_context,
        "question": query
    })
    
    if hasattr(enhanced_response, 'content'):
        enhanced_response = enhanced_response.content
    
    if debug:
        print("Recursive retrieval-generation complete")
    
    return enhanced_response

In [58]:
from IPython.display import display, Markdown

display(Markdown(get_answer_with_recursive_generation("How to delete a policy?")))

/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args

  Found 1 previous chunks and 2 next chunks


To delete a policy block using the CyberArk API, you can follow the steps outlined below:

### Endpoint Information
- **HTTP Method:** POST
- **Path:** `/Policy/DeletePolicyBlock`

### Authentication
- You must authenticate using **bearer token security** to access this endpoint.

### Required Parameters
- **path** (string): This is the specific policy block that you want to delete. This field is required.

### Request Structure
The request must be sent in JSON format with the following structure:

```json
{
  "path": "string_value"
}
```

**Example Request:**
```json
{
  "path": "policy/block/123"
}
```

### Response Structure
Upon a successful deletion, you will receive a response with the following structure:

- **Status Code:** `200`
- **Response Body Properties:**
  - **Result** (boolean): Indicates if the policy block deletion succeeded.
  - **Error** (object): Contains error message text on failure, may be null.

**Example Response:**
```json
{
  "Result": true,
  "Error": {}
}
```

### Error Handling
If the deletion fails, the `Error` object will contain a message detailing the issue. For example:

```json
{
  "Result": false,
  "Error": {
    "message": "Policy block not found."
  }
}
```

### Summary
To delete a policy block:
1. Make a POST request to `/Policy/DeletePolicyBlock`.
2. Include the required `path` parameter in the request body.
3. Ensure you are authenticated with a bearer token.
4. Check the response to confirm if the deletion was successful or if there was an error.

If you have any further questions or need additional assistance, feel free to ask!